In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import tqdm

In [3]:
data_dir = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/Dataset_wiki'

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

In [5]:
train_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=data_dir + '/test', transform=transform)


In [6]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 3: Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=3, num_classes=2).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

Epoch [1/10], Loss: 0.3947
Validation Accuracy: 0.9452
Epoch [2/10], Loss: 0.1146
Validation Accuracy: 0.9620
Epoch [3/10], Loss: 0.0889
Validation Accuracy: 0.9623
Epoch [4/10], Loss: 0.0720
Validation Accuracy: 0.9752
Epoch [5/10], Loss: 0.0660
Validation Accuracy: 0.9633
Epoch [6/10], Loss: 0.0588
Validation Accuracy: 0.9812
Epoch [7/10], Loss: 0.0539
Validation Accuracy: 0.9772
Epoch [8/10], Loss: 0.0514
Validation Accuracy: 0.9820
Epoch [9/10], Loss: 0.0525
Validation Accuracy: 0.9777
Epoch [10/10], Loss: 0.0490
Validation Accuracy: 0.9790


In [8]:
import torch
import torchvision.models as models

model_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/model/wiki_model.pth"

torch.save(model.state_dict(), model_path)


In [25]:
print(model)
#print(image_tensor.shape)


CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu5): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


In [65]:
image_path = "/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/sample_human.png"

In [66]:
image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0)  

In [67]:
image_tensor = image_tensor.to(device)

In [68]:
with torch.no_grad():
    output = model(image_tensor)

# Convert the output probabilities to predicted class
_, predicted = torch.max(output, 1)

In [69]:
print(predicted)

tensor([0], device='cuda:0')


In [70]:
print(output)

tensor([[ 1.6003, -1.3525]], device='cuda:0')
